# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## install segmentation-models
this package containes segmentation models predefined with pretrained weights

In [2]:
!pip install segmentation-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 6.8 MB/s 


### Loading Dataset

In [3]:

!pip3 install tensorflow==2.2
!pip3 install keras==2.3.1
!pip3 install -U segmentation-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 516.2 MB 4.4 kB/s 
     |████████████████████████████████| 454 kB 67.6 MB/s 
     |████████████████████████████████| 26.1 MB 1.3 MB/s 
     |████████████████████████████████| 2.9 MB 39.8 MB/s 
     |████████████████████████████████| 3.0 MB 63.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Unins

In [5]:
import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [6]:
SIZE_Y = 256
SIZE_X = 256

# Load Train

In [21]:
img_dir = os.listdir("/content/drive/MyDrive/Dataset/INbreast/dataset/breast_tight/train/a/")
train_images = []
for index,img_id in enumerate(img_dir):
  img_path = f"/content/drive/MyDrive/Dataset/INbreast/dataset/breast_tight/train/a/{img_id}"
  img = cv2.imread(img_path, cv2.IMREAD_COLOR) 
  img = cv2.resize(img, (SIZE_Y, SIZE_X))
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  train_images.append(img)
  if index == 149:
    break

In [22]:
train_images = np.array(train_images)

In [23]:
train_images.shape

(92, 256, 256, 3)

In [24]:
img_dir = os.listdir("/content/drive/MyDrive/Dataset/INbreast/dataset/breast_tight/train/b/")
train_masks = []
for index,img_id in enumerate(img_dir):
  img_path = f"/content/drive/MyDrive/Dataset/INbreast/dataset/breast_tight/train/b/{img_id}"
  img = cv2.imread(img_path, cv2.IMREAD_COLOR) 
  img = cv2.resize(img, (SIZE_Y, SIZE_X))
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  train_masks.append(img)
  if index == 149:
    break

In [25]:
train_masks = np.array(train_masks)

In [26]:
train_masks.shape

(92, 256, 256, 3)

# Load Test

In [27]:
img_dir = os.listdir("/content/drive/MyDrive/Dataset/INbreast/dataset/breast_tight/test/a/")
test_images = []
for index,img_id in enumerate(img_dir):
  img_path = f"/content/drive/MyDrive/Dataset/INbreast/dataset/breast_tight/test/a/{img_id}"
  img = cv2.imread(img_path, cv2.IMREAD_COLOR) 
  img = cv2.resize(img, (SIZE_Y, SIZE_X))
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  test_images.append(img)
  if index == 149:
    break

In [28]:
test_images = np.array(test_images)

In [29]:
test_images.shape

(23, 256, 256, 3)

In [30]:
img_dir = os.listdir("/content/drive/MyDrive/Dataset/INbreast/dataset/breast_tight/test/b/")
test_masks = []
for index,img_id in enumerate(img_dir):
  img_path = f"/content/drive/MyDrive/Dataset/INbreast/dataset/breast_tight/test/b/{img_id}"
  img = cv2.imread(img_path, cv2.IMREAD_COLOR) 
  img = cv2.resize(img, (SIZE_Y, SIZE_X))
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  test_masks.append(img)
  if index == 149:
    break

In [31]:
test_masks = np.array(test_masks)

In [32]:
test_masks.shape

(23, 256, 256, 3)

#### preprocess input

In [33]:
train_images = preprocess_input(train_images)
test_images = preprocess_input(test_images)

In [34]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

In [35]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 3

## Define LinkNet
### 20,325,427 Parameters

In [38]:
from segmentation_models import Linknet

model = Linknet(encoder_weights='imagenet', encoder_freeze=True, input_shape=(256, 256,3),classes=3)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 256, 256, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 256, 256, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 128, 128, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

In [39]:
history=model.fit(train_images, train_masks, batch_size=2, epochs=300, verbose=1, validation_data=(test_images, test_masks))

Train on 92 samples, validate on 23 samples
Epoch 1/300
92/92 [==============================] - 13s 145ms/step - loss: -279.4435 - accuracy: 0.0623 - val_loss: -7748.2751 - val_accuracy: 0.0226
Epoch 2/300
92/92 [==============================] - 3s 36ms/step - loss: -1233.0832 - accuracy: 0.0240 - val_loss: -9286.3392 - val_accuracy: 0.0226
Epoch 3/300
92/92 [==============================] - 3s 35ms/step - loss: -2681.8421 - accuracy: 0.0178 - val_loss: -2665.9602 - val_accuracy: 0.0226
Epoch 4/300
92/92 [==============================] - 3s 36ms/step - loss: -4497.6009 - accuracy: 0.0178 - val_loss: -5742.8815 - val_accuracy: 0.0226
Epoch 5/300
92/92 [==============================] - 3s 36ms/step - loss: -6649.5443 - accuracy: 0.0178 - val_loss: -10231.3459 - val_accuracy: 0.0226
Epoch 6/300
92/92 [==============================] - 3s 36ms/step - loss: -9157.9130 - accuracy: 0.0178 - val_loss: -13667.8236 - val_accuracy: 0.0226
Epoch 7/300
92/92 [==============================] - 

In [ ]:
#accuracy = model.evaluate(x_val, y_val)
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('tight_LinkNet_Loss.jpg',dpi=400)

In [ ]:
#accuracy = model.evaluate(x_val, y_val)
#plot the training and validation accuracy and loss at each epoch
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.grid(True)
plt.savefig('tight_LinkNet_Accuracy.jpg',dpi=400)

In [42]:
model.save('/content/drive/MyDrive/Dataset/INbreast/checkpoint/breast_full/INBREAST_tight_LinkNet.h5')

### Use trained model to predict masks (zip -> download)

In [43]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/Dataset/INbreast/checkpoint/breast_full/INBREAST_tight_LinkNet.h5', compile=False)

In [44]:
prediction = model.predict(test_images)

In [45]:
prediction.shape

(23, 256, 256, 3)

In [46]:
img_dir = os.listdir("/content/drive/MyDrive/Dataset/INbreast/dataset/breast_tight/test/b/")
img_names = []
for index,img_id in enumerate(img_dir):
    img_names.append(img_id)

i = 0
for img_msk in prediction:
    plt.imsave(f"/content/sample_data/Result/{img_names[i]}",img_msk,dpi=300)
    i+=1

In [47]:
!zip -r /content/tight_Link.zip /content/sample_data/Result

  adding: content/sample_data/Result/ (stored 0%)
  adding: content/sample_data/Result/20587758.jpeg (deflated 61%)
  adding: content/sample_data/Result/20587994.jpeg (deflated 60%)
  adding: content/sample_data/Result/2_51049107.jpeg (deflated 60%)
  adding: content/sample_data/Result/20587612.jpeg (deflated 59%)
  adding: content/sample_data/Result/2_20586908.jpeg (deflated 59%)
  adding: content/sample_data/Result/20587664.jpeg (deflated 60%)
  adding: content/sample_data/Result/2_22427840.jpeg (deflated 60%)
  adding: content/sample_data/Result/20587902.jpeg (deflated 59%)
  adding: content/sample_data/Result/2_22580341.jpeg (deflated 61%)
  adding: content/sample_data/Result/20586934.jpeg (deflated 61%)
  adding: content/sample_data/Result/2_22579730.jpeg (deflated 60%)
  adding: content/sample_data/Result/2_22580367.jpeg (deflated 59%)
  adding: content/sample_data/Result/20586908.jpeg (deflated 59%)
  adding: content/sample_data/Result/2_20586960.jpeg (deflated 60%)
  adding: co